In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
import pandas as pd
import seaborn as sns
import anndata
import scanpy as sc
import genetools
from covid_serology import config, helpers

# Load data

In [2]:
infection_df = pd.read_csv(
    f"{config.paths.data_dir}/variant_infections_seropositives.plate11.csv"
)
infection_df.head()

,ID,variant,vaccine,vaccinated,mAb_cocktail,Epsilon_ECL,Epsilon_AU,Beta_ECL,Beta_AU,Iota_ECL,...,Alpha_ECL,Alpha_AU,P.3_ECL,P.3_AU,Kappa_ECL,Kappa_AU,Delta_ECL,Delta_AU,Wuhan_ECL,Wuhan_AU
0,77,delta,Not vaccinated,NaN,yes,1746523,6.171326e+06,804106,3.451997e+06,1366582,...,1394052,4.925397e+06,756470,4.400233e+06,1750546,7.561206e+06,1602908,4.913945e+06,1790094,6.236417e+06
1,101,delta,Not vaccinated,NaN,yes,183877,5.722873e+05,74056,2.656252e+05,127515,...,127502,3.861109e+05,74434,3.392012e+05,156077,5.815294e+05,169561,4.623276e+05,182724,5.653562e+05
2,2,alpha,Not vaccinated,NaN,yes,425442,1.436242e+06,182480,5.354031e+05,324485,...,329195,1.020293e+06,201313,6.878624e+05,405399,1.588069e+06,405975,1.102106e+06,422598,1.379488e+06
3,7,alpha,Not vaccinated,no,no,23775,8.571427e+03,7342,2.238512e+03,7236,...,96398,3.302893e+04,9838,4.497432e+03,13885,5.766121e+03,18436,5.271091e+03,37426,1.372663e+04
4,8,alpha,Not vaccinated,no,no,330037,1.126644e+05,148888,4.635208e+04,110146,...,694244,2.352883e+05,89004,4.026055e+04,173923,7.151129e+04,284913,7.637192e+04,382711,1.332992e+05


In [3]:
# prefix so these IDs are unique from other datasets
infection_df["ID"] = "VariantInfection_" + infection_df["ID"].astype(str)

In [4]:
infection_df.groupby(["vaccinated", "vaccine"]).size()

vaccinated  vaccine       
no          Not vaccinated    25
yes         Janssen            2
            Moderna            6
            Pfizer             8
dtype: int64

In [5]:
infection_df["mAb_cocktail"].fillna("no", inplace=True)

In [6]:
# Remove mAb-treated patients - the external antibodies may interfere with our measurements
infection_df = infection_df[
    (infection_df["vaccinated"] != "mAb") & (infection_df["mAb_cocktail"] != "yes")
].copy()

In [7]:
assert not infection_df["vaccinated"].isna().any()
assert not infection_df["vaccine"].isna().any()
assert not infection_df["mAb_cocktail"].isna().any()

In [8]:
infection_df.groupby(["vaccinated", "vaccine", "mAb_cocktail"]).size()

vaccinated  vaccine         mAb_cocktail
no          Not vaccinated  no              25
yes         Janssen         no               2
            Moderna         no               6
            Pfizer          no               8
dtype: int64

In [9]:
infection_df.head()

,ID,variant,vaccine,vaccinated,mAb_cocktail,Epsilon_ECL,Epsilon_AU,Beta_ECL,Beta_AU,Iota_ECL,...,Alpha_ECL,Alpha_AU,P.3_ECL,P.3_AU,Kappa_ECL,Kappa_AU,Delta_ECL,Delta_AU,Wuhan_ECL,Wuhan_AU
3,VariantInfection_7,alpha,Not vaccinated,no,no,23775,8571.427475,7342,2238.511553,7236,...,96398,33028.931300,9838,4497.432041,13885,5766.120823,18436,5271.090797,37426,13726.629290
4,VariantInfection_8,alpha,Not vaccinated,no,no,330037,112664.426300,148888,46352.076440,110146,...,694244,235288.347900,89004,40260.552060,173923,71511.291520,284913,76371.918560,382711,133299.236600
5,VariantInfection_10,alpha,Not vaccinated,no,no,136856,47598.261790,87724,27236.876620,95013,...,198950,67881.729890,88086,39847.583800,119593,49030.680980,136376,37215.301220,171850,60942.653910
6,VariantInfection_11,delta,Not vaccinated,no,no,78325,27564.097630,13134,4030.672198,20757,...,31604,10891.239660,10128,4629.803618,53180,21840.335700,137252,37448.532580,45518,16624.604140
9,VariantInfection_33,delta,Not vaccinated,no,no,11467,4188.274494,526,136.977506,2262,...,1214,403.513183,440,176.869180,7633,3181.325633,25926,7357.724867,3928,1494.242724


In [10]:
infection_df.columns

Index(['ID', 'variant', 'vaccine', 'vaccinated', 'mAb_cocktail', 'Epsilon_ECL',
       'Epsilon_AU', 'Beta_ECL', 'Beta_AU', 'Iota_ECL', 'Iota_AU', 'Gamma_ECL',
       'Gamma_AU', 'B.1.526.2_ECL', 'B.1.526.2_AU', 'Alpha_ECL', 'Alpha_AU',
       'P.3_ECL', 'P.3_AU', 'Kappa_ECL', 'Kappa_AU', 'Delta_ECL', 'Delta_AU',
       'Wuhan_ECL', 'Wuhan_AU'],
      dtype='object')

In [11]:
infection_df["variant"].value_counts()

delta    34
alpha     7
Name: variant, dtype: int64

In [12]:
infection_df["variant"] = infection_df["variant"].str.title()

In [13]:
infection_df["variant"].value_counts()

Delta    34
Alpha     7
Name: variant, dtype: int64

In [14]:
infection_df["vaccine"].value_counts()

Not vaccinated    25
Pfizer             8
Moderna            6
Janssen            2
Name: vaccine, dtype: int64

In [15]:
infection_df["vaccine"] = infection_df["vaccine"].replace(
    {"Pfizer": "mRNA", "Moderna": "mRNA"}
)

In [16]:
infection_df["vaccine"].value_counts()

Not vaccinated    25
mRNA              14
Janssen            2
Name: vaccine, dtype: int64

In [17]:
measurement_cols = infection_df.columns[infection_df.columns.str.contains("AU")]
helpers.confirm_all_measurement_columns_are_present(measurement_cols)
measurement_cols

Index(['Epsilon_AU', 'Beta_AU', 'Iota_AU', 'Gamma_AU', 'B.1.526.2_AU',
       'Alpha_AU', 'P.3_AU', 'Kappa_AU', 'Delta_AU', 'Wuhan_AU'],
      dtype='object')

In [18]:
# only one sample per patient
assert all(infection_df["ID"].value_counts() == 1)

# reformat

In [19]:
# Separate into groups by variant and by vaccination status if any

infection_df_obs = infection_df[
    [
        "ID",
        "variant",
        "vaccine",
    ]
].copy()

infection_df_obs["Status"] = "Variant Infection" + " - " + infection_df_obs["variant"]
# add vaccine info suffix if vaccinated
infection_df_obs.loc[infection_df_obs["vaccine"] != "Not vaccinated", "Status"] += (
    " - "
    + infection_df_obs.loc[infection_df_obs["vaccine"] != "Not vaccinated", "vaccine"]
    + " vaccinated"
)

# capitalize columns for consistency - only the first letter
# and set index
infection_df_obs = infection_df_obs.rename(
    columns=lambda s: s[0].upper() + s[1:]
).set_index("ID")

# anndata wants string index
infection_df_obs.index = infection_df_obs.index.astype(str)

infection_df_obs

,Variant,Vaccine,Status
ID,,,
VariantInfection_7,Alpha,Not vaccinated,Variant Infection - Alpha
VariantInfection_8,Alpha,Not vaccinated,Variant Infection - Alpha
VariantInfection_10,Alpha,Not vaccinated,Variant Infection - Alpha
VariantInfection_11,Delta,Not vaccinated,Variant Infection - Delta
VariantInfection_33,Delta,Not vaccinated,Variant Infection - Delta
VariantInfection_34,Delta,Not vaccinated,Variant Infection - Delta
VariantInfection_38,Delta,Not vaccinated,Variant Infection - Delta
VariantInfection_43,Delta,Not vaccinated,Variant Infection - Delta
VariantInfection_45,Delta,Not vaccinated,Variant Infection - Delta


In [20]:
# extract AU measurement cols, and set index to match obs
infection_df_X = (
    infection_df[measurement_cols]
    .rename(columns=lambda col: col.replace("_AU", ""))
    .set_index(infection_df_obs.index)
)

infection_df_X

,Epsilon,Beta,Iota,Gamma,B.1.526.2,Alpha,P.3,Kappa,Delta,Wuhan
ID,,,,,,,,,,
VariantInfection_7,8.571427e+03,2.238512e+03,2.916765e+03,4.671838e+03,1.209455e+04,3.302893e+04,4.497432e+03,5.766121e+03,5.271091e+03,1.372663e+04
VariantInfection_8,1.126644e+05,4.635208e+04,4.371507e+04,8.709362e+04,1.400774e+05,2.352883e+05,4.026055e+04,7.151129e+04,7.637192e+04,1.332992e+05
VariantInfection_10,4.759826e+04,2.723688e+04,3.775660e+04,4.178509e+04,5.274343e+04,6.788173e+04,3.984758e+04,4.903068e+04,3.721530e+04,6.094265e+04
VariantInfection_11,2.756410e+04,4.030672e+03,8.344526e+03,6.929228e+03,1.334704e+04,1.089124e+04,4.629804e+03,2.184034e+04,3.744853e+04,1.662460e+04
VariantInfection_33,4.188274e+03,1.369775e+02,8.979137e+02,2.419405e+02,1.248200e+03,4.035132e+02,1.768692e+02,3.181326e+03,7.357725e+03,1.494243e+03
VariantInfection_34,1.297544e+05,1.874687e+04,3.720103e+04,3.934789e+04,6.027608e+04,4.697583e+04,1.526849e+04,1.156698e+05,2.001354e+05,6.593848e+04
VariantInfection_38,8.033188e+03,6.275274e+02,1.610966e+03,1.015129e+03,2.550453e+03,1.511530e+03,6.357270e+02,6.505540e+03,1.492881e+04,2.688440e+03
VariantInfection_43,5.595303e+03,7.913871e+02,1.213401e+03,1.265423e+03,1.951490e+03,1.494494e+03,6.463956e+02,3.986886e+03,8.078818e+03,2.201217e+03
VariantInfection_45,1.198095e+04,1.586022e+03,3.273222e+03,3.273990e+03,6.307536e+03,2.310618e+03,6.603088e+02,1.410956e+04,5.765910e+04,6.330453e+03


In [21]:
adata = anndata.AnnData(X=infection_df_X, obs=infection_df_obs)
adata

AnnData object with n_obs × n_vars = 41 × 10
    obs: 'Variant', 'Vaccine', 'Status'

In [22]:
adata.obs

,Variant,Vaccine,Status
ID,,,
VariantInfection_7,Alpha,Not vaccinated,Variant Infection - Alpha
VariantInfection_8,Alpha,Not vaccinated,Variant Infection - Alpha
VariantInfection_10,Alpha,Not vaccinated,Variant Infection - Alpha
VariantInfection_11,Delta,Not vaccinated,Variant Infection - Delta
VariantInfection_33,Delta,Not vaccinated,Variant Infection - Delta
VariantInfection_34,Delta,Not vaccinated,Variant Infection - Delta
VariantInfection_38,Delta,Not vaccinated,Variant Infection - Delta
VariantInfection_43,Delta,Not vaccinated,Variant Infection - Delta
VariantInfection_45,Delta,Not vaccinated,Variant Infection - Delta


In [23]:
adata.var

""
Epsilon
Beta
Iota
Gamma
B.1.526.2
Alpha
P.3
Kappa
Delta
Wuhan


In [24]:
adata.var["virus"] = adata.var_names
adata.var["target"] = "RBD"
adata.var["variant_plate_type"] = "Variant"
adata.var["antibody"] = "IgG"
adata.var

,virus,target,variant_plate_type,antibody
Epsilon,Epsilon,RBD,Variant,IgG
Beta,Beta,RBD,Variant,IgG
Iota,Iota,RBD,Variant,IgG
Gamma,Gamma,RBD,Variant,IgG
B.1.526.2,B.1.526.2,RBD,Variant,IgG
Alpha,Alpha,RBD,Variant,IgG
P.3,P.3,RBD,Variant,IgG
Kappa,Kappa,RBD,Variant,IgG
Delta,Delta,RBD,Variant,IgG
Wuhan,Wuhan,RBD,Variant,IgG


In [25]:
adata.X

array([[8.57142773e+03, 2.23851147e+03, 2.91676514e+03, 4.67183838e+03,
        1.20945449e+04, 3.30289297e+04, 4.49743213e+03, 5.76612061e+03,
        5.27109082e+03, 1.37266289e+04],
       [1.12664430e+05, 4.63520781e+04, 4.37150703e+04, 8.70936172e+04,
        1.40077438e+05, 2.35288344e+05, 4.02605508e+04, 7.15112891e+04,
        7.63719219e+04, 1.33299234e+05],
       [4.75982617e+04, 2.72368770e+04, 3.77566016e+04, 4.17850859e+04,
        5.27434297e+04, 6.78817266e+04, 3.98475820e+04, 4.90306797e+04,
        3.72153008e+04, 6.09426523e+04],
       [2.75640977e+04, 4.03067212e+03, 8.34452539e+03, 6.92922803e+03,
        1.33470439e+04, 1.08912393e+04, 4.62980371e+03, 2.18403359e+04,
        3.74485312e+04, 1.66246035e+04],
       [4.18827441e+03, 1.36977509e+02, 8.97913696e+02, 2.41940536e+02,
        1.24820032e+03, 4.03513184e+02, 1.76869186e+02, 3.18132568e+03,
        7.35772510e+03, 1.49424268e+03],
       [1.29754398e+05, 1.87468691e+04, 3.72010273e+04, 3.93478945e+04,
   

In [26]:
adata

AnnData object with n_obs × n_vars = 41 × 10
    obs: 'Variant', 'Vaccine', 'Status'
    var: 'virus', 'target', 'variant_plate_type', 'antibody'

In [27]:
adata.var["timepoint"] = "day 28 / week 4"
adata.var

,virus,target,variant_plate_type,antibody,timepoint
Epsilon,Epsilon,RBD,Variant,IgG,day 28 / week 4
Beta,Beta,RBD,Variant,IgG,day 28 / week 4
Iota,Iota,RBD,Variant,IgG,day 28 / week 4
Gamma,Gamma,RBD,Variant,IgG,day 28 / week 4
B.1.526.2,B.1.526.2,RBD,Variant,IgG,day 28 / week 4
Alpha,Alpha,RBD,Variant,IgG,day 28 / week 4
P.3,P.3,RBD,Variant,IgG,day 28 / week 4
Kappa,Kappa,RBD,Variant,IgG,day 28 / week 4
Delta,Delta,RBD,Variant,IgG,day 28 / week 4
Wuhan,Wuhan,RBD,Variant,IgG,day 28 / week 4


In [28]:
# create combined name
adata.var["combined_name"] = adata.var.apply("_".join, axis=1)
adata.var = adata.var.set_index("combined_name")

In [29]:
adata.var

,virus,target,variant_plate_type,antibody,timepoint
combined_name,,,,,
Epsilon_RBD_Variant_IgG_day 28 / week 4,Epsilon,RBD,Variant,IgG,day 28 / week 4
Beta_RBD_Variant_IgG_day 28 / week 4,Beta,RBD,Variant,IgG,day 28 / week 4
Iota_RBD_Variant_IgG_day 28 / week 4,Iota,RBD,Variant,IgG,day 28 / week 4
Gamma_RBD_Variant_IgG_day 28 / week 4,Gamma,RBD,Variant,IgG,day 28 / week 4
B.1.526.2_RBD_Variant_IgG_day 28 / week 4,B.1.526.2,RBD,Variant,IgG,day 28 / week 4
Alpha_RBD_Variant_IgG_day 28 / week 4,Alpha,RBD,Variant,IgG,day 28 / week 4
P.3_RBD_Variant_IgG_day 28 / week 4,P.3,RBD,Variant,IgG,day 28 / week 4
Kappa_RBD_Variant_IgG_day 28 / week 4,Kappa,RBD,Variant,IgG,day 28 / week 4
Delta_RBD_Variant_IgG_day 28 / week 4,Delta,RBD,Variant,IgG,day 28 / week 4


In [30]:
adata

AnnData object with n_obs × n_vars = 41 × 10
    obs: 'Variant', 'Vaccine', 'Status'
    var: 'virus', 'target', 'variant_plate_type', 'antibody', 'timepoint'

In [31]:
# filter down variant infection types
adata.obs["Status"].value_counts()

Variant Infection - Delta                         21
Variant Infection - Delta - mRNA vaccinated       12
Variant Infection - Alpha                          4
Variant Infection - Alpha - mRNA vaccinated        2
Variant Infection - Delta - Janssen vaccinated     1
Variant Infection - Alpha - Janssen vaccinated     1
Name: Status, dtype: int64

In [32]:
# filter down variant infection types
adata = adata[
    adata.obs["Status"].isin(
        [
            "Variant Infection - Delta",
            "Variant Infection - Delta - mRNA vaccinated",
            "Variant Infection - Alpha",
        ]
    )
]

In [33]:
adata

View of AnnData object with n_obs × n_vars = 37 × 10
    obs: 'Variant', 'Vaccine', 'Status'
    var: 'virus', 'target', 'variant_plate_type', 'antibody', 'timepoint'

In [34]:
adata.write(f"{config.paths.generated_data_dir}/partial.variant_infections.h5")

/users/maximz/anaconda3/envs/serology-env/lib/python3.7/site-packages/anndata/_core/anndata.py:1237: ImplicitModificationWarning: Initializing view as actual.
  "Initializing view as actual.", ImplicitModificationWarning
Trying to set attribute `.obs` of view, copying.


... storing 'Variant' as categorical


Trying to set attribute `.obs` of view, copying.


... storing 'Vaccine' as categorical


Trying to set attribute `.obs` of view, copying.


...

... storing 'variant_plate_type' as categorical


Trying to set attribute `.var` of view, copying.


... storing 'antibody' as categorical


Trying to set attribute `.var` of view, copying.


... storing 'timepoint' as categorical
